# Training Policy

##### This module deals with training any policy and serialize and store the model

In [33]:
import pandas as pd
import numpy as np
import pickle
from my_packages.policy_train import PolicyTrain
from my_packages.environment import StockEnv

In [34]:
#this contains the path where the policy thaT is to be trained is stored
policy_path = "../model/model7inputs"
#now the path to data on which the policy would be trained
data_path = "./data/AAPL-feb-25.csv"

In [35]:
file = open(policy_path , 'rb')
policy = pickle.load(file)
file.close()

In [36]:
# the feature that are required for this model is :
policy.data_features

['c-o/o', 'h-l/o', 'h-o/o', 'o-l/o', 'c', 'vol', 'avg_purchase_price']

In [37]:
df = pd.read_csv(data_path)
df.head()

,c,h,l,n,o,t,v,vw
0,234.9493,234.9493,234.9200,16,234.9200,2025-02-01T00:00:00Z,380,234.939340
1,235.0695,235.0695,234.9204,35,234.9204,2025-02-01T00:01:00Z,3894,235.058980
2,234.9220,234.9220,234.9220,20,234.9220,2025-02-01T00:02:00Z,184,234.922000
3,234.9201,234.9980,234.9201,20,234.9980,2025-02-01T00:03:00Z,946,234.947742
4,234.9323,234.9323,234.9323,18,234.9323,2025-02-01T00:04:00Z,255,234.932300


for scalling purposes we want to scale the ratios multipling them by 100 , the close price is scaled down by 1e4 and the volume will be scaled down by a fraction of 1e5 

In [38]:
training_df = pd.DataFrame()
training_df.insert(0,'co',((df['c'] - df['o'] )*100/ df['o']), allow_duplicates=True)
training_df.insert(1,'hl',((df['h'] - df['l'] )*100/ df['o']), allow_duplicates=True)
training_df.insert(2,'ho',((df['h'] - df['o'] )*100/ df['o']), allow_duplicates=True)
training_df.insert(3,'ol',((df['o'] - df['l'] )*100/ df['o']), allow_duplicates=True)
training_df.insert(4,'c',(df['c']/10000),allow_duplicates=True)
training_df.insert(5,'vol',(df['v']/100000), allow_duplicates=True)
training_df.head()

,co,hl,ho,ol,c,vol
0,0.012472,0.012472,0.012472,0.000000,0.023495,0.00380
1,0.063468,0.063468,0.063468,0.000000,0.023507,0.03894
2,0.000000,0.000000,0.000000,0.000000,0.023492,0.00184
3,-0.033149,0.033149,0.000000,0.033149,0.023492,0.00946
4,0.000000,0.000000,0.000000,0.000000,0.023493,0.00255


In [39]:
print("max : ",training_df.max(axis=0))
print("min : ",training_df.min(axis=0))
print("avg : ",training_df.mean(axis=0))



max :  co       0.995443
hl       3.247154
ho       2.523604
ol       1.620441
c        0.024860
vol    134.577090
dtype: float64
min :  co    -1.397685
hl     0.000000
ho     0.000000
ol     0.000000
c      0.022575
vol    0.001000
dtype: float64
avg :  co     0.001120
hl     0.058968
ho     0.030308
ol     0.028660
c      0.023567
vol    0.584269
dtype: float64


In [40]:
X = training_df.to_numpy()
X.shape

(10000, 6)

In [41]:
env = StockEnv()
env.set_env_data(X)

In [42]:
training_tool = PolicyTrain(policy,env,epsilon=0.5,gamma=0.95)

In [43]:
training_tool.episode_train(batch_size=100)
policy.training_log.append(data_path)

In [44]:
prediction = policy.predict(np.append(X[9999,:] , 1))
print(prediction)
np.argmax(prediction[0])

[[-5.27828753e-06 -2.92972789e-06 -2.83964932e-06]]


2

In [45]:
file = open(policy_path,'wb')
pickle.dump(policy,file)
file.close()